<a href="https://colab.research.google.com/github/KCREEK/Assembly/blob/main/blockchain_simulation_cu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile blockchain_simulation.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define MAX_BLOCKS 1024 // Max blocks for this example
#define THREADS_PER_BLOCK 256 // Max threads per block

// Structure to define a block in the blockchain
struct Block {
    int blockID;
    long timestamp;
    int transactionCount;
    int previousHash;
    int currentHash;
};

// Kernel to calculate block hashes
__global__ void calculateHash(Block *blocks, int *hashes, int numBlocks) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < numBlocks) {
        hashes[index] = blocks[index].previousHash ^ blocks[index].transactionCount + blocks[index].timestamp;
    }
}

// Kernel to count the number of blocks in the blockchain
__global__ void countBlocks(int *count, int numBlocks) {
    __shared__ int countCache[THREADS_PER_BLOCK];
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    countCache[threadIdx.x] = (index < numBlocks) ? 1 : 0;
    __syncthreads();

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        if (threadIdx.x < offset) {
            countCache[threadIdx.x] += countCache[threadIdx.x + offset];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(count, countCache[0]);
    }
}

// Function to initialize a dataset of blocks
void initializeBlocks(Block *blocks, int numBlocks) {
    for (int i = 0; i < numBlocks; i++) {
        blocks[i].blockID = i;
        blocks[i].timestamp = 1620000000 + i * 10;
        blocks[i].transactionCount = (i % 10) + 1;
        blocks[i].previousHash = i * 31;
        blocks[i].currentHash = 0;
    }
}

// Function to print the blocks
void printBlocks(const Block *blocks, int numBlocks) {
    printf("BlockID\tTimestamp\tTransactions\tPrevHash\tCurrHash\n");
    for (int i = 0; i < numBlocks; i++) {
        printf("%d\t%ld\t%d\t%d\t%d\n",
               blocks[i].blockID,
               blocks[i].timestamp,
               blocks[i].transactionCount,
               blocks[i].previousHash,
               blocks[i].currentHash);
    }
}

int main() {
    int numBlocks = MAX_BLOCKS;
    int *d_count;
    int host_count = 0;
    Block *blocks = (Block*)malloc(numBlocks * sizeof(Block));

    initializeBlocks(blocks, numBlocks);

    Block *d_blocks;
    int *d_hashes;
    cudaMalloc((void**)&d_blocks, numBlocks * sizeof(Block));
    cudaMalloc((void**)&d_hashes, numBlocks * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));



    cudaMemcpy(d_blocks, blocks, numBlocks * sizeof(Block), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &host_count, sizeof(int), cudaMemcpyHostToDevice);

    int blocksPerGrid = (numBlocks + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    calculateHash<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_blocks, d_hashes, numBlocks);
    cudaDeviceSynchronize();

    countBlocks<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_count, numBlocks);
    cudaDeviceSynchronize();

    cudaMemcpy(&host_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(blocks, d_blocks, numBlocks * sizeof(Block), cudaMemcpyDeviceToHost);

    printf("Blockchain Simulation:\n");
    printBlocks(blocks, numBlocks);


    cudaFree(d_blocks);
    cudaFree(d_hashes);
    cudaFree(d_count);
    free(blocks);

    return 0;
}


Overwriting blockchain_simulation.cu


In [ ]:
%%writefile blockchain_simulation.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define MAX_BLOCKS 1024 // Max blocks for this example
#define THREADS_PER_BLOCK 256 // Max threads per block

// Structure to define a block in the blockchain
struct Block {
    int blockID;
    long timestamp;
    int transactionCount;
    int previousHash;
    int currentHash;
};

// Kernel to calculate block hashes
__global__ void calculateHash(Block *blocks, int *hashes, int numBlocks) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < numBlocks) {
        hashes[index] = blocks[index].previousHash ^ blocks[index].transactionCount + blocks[index].timestamp;
    }
}

// Kernel to count the number of blocks in the blockchain
__global__ void countBlocks(int *count, int numBlocks) {
    __shared__ int countCache[THREADS_PER_BLOCK];
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    countCache[threadIdx.x] = (index < numBlocks) ? 1 : 0;
    __syncthreads();

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        if (threadIdx.x < offset) {
            countCache[threadIdx.x] += countCache[threadIdx.x + offset];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(count, countCache[0]);
    }
}

// Function to initialize a dataset of blocks
void initializeBlocks(Block *blocks, int numBlocks) {
    for (int i = 0; i < numBlocks; i++) {
        blocks[i].blockID = i;
        blocks[i].timestamp = 1620000000 + i * 10;
        blocks[i].transactionCount = (i % 10) + 1;
        blocks[i].previousHash = i * 31;
        blocks[i].currentHash = 0;
    }
}

// Function to print the blocks
void printBlocks(const Block *blocks, int numBlocks) {
    printf("BlockID\tTimestamp\tTransactions\tPrevHash\tCurrHash\n");
    for (int i = 0; i < numBlocks; i++) {
        printf("%d\t%ld\t%d\t%d\t%d\n",
               blocks[i].blockID,
               blocks[i].timestamp,
               blocks[i].transactionCount,
               blocks[i].previousHash,
               blocks[i].currentHash);
    }
}

int main() {
    int numBlocks = MAX_BLOCKS;
    int *d_count;
    int host_count = 0;
    Block *blocks = (Block*)malloc(numBlocks * sizeof(Block));

    initializeBlocks(blocks, numBlocks);

    Block *d_blocks;
    int *d_hashes;
    cudaMalloc((void**)&d_blocks, numBlocks * sizeof(Block));
    cudaMalloc((void**)&d_hashes, numBlocks * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));



    cudaMemcpy(d_blocks, blocks, numBlocks * sizeof(Block), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &host_count, sizeof(int), cudaMemcpyHostToDevice);

    int blocksPerGrid = (numBlocks + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    calculateHash<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_blocks, d_hashes, numBlocks);
    cudaDeviceSynchronize();

    countBlocks<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_count, numBlocks);
    cudaDeviceSynchronize();

    cudaMemcpy(&host_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(blocks, d_blocks, numBlocks * sizeof(Block), cudaMemcpyDeviceToHost);

    printf("Blockchain Simulation:\n");
    printBlocks(blocks, numBlocks);

     FILE *outputFile = fopen("output.txt", "w");
           if (outputFile) {
    fprintf(outputFile, "Blockchain Simulation:\n");
    for (int i = 0; i < numBlocks; i++) {
        fprintf(outputFile, "%d\t%ld\t%d\t%d\t%d\n", blocks[i].blockID, blocks[i].timestamp, blocks[i].transactionCount, blocks[i].previousHash, blocks[i].currentHash);
        }
    fprintf(outputFile, "\nTotal number of blocks: %d\n", host_count);
    fclose(outputFile);
     }


    cudaFree(d_blocks);
    cudaFree(d_hashes);
    cudaFree(d_count);
    free(blocks);

    return 0;
}


Overwriting blockchain_simulation.cu


In [14]:
%%writefile blockchain_simulation.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define MAX_BLOCKS 1024 // Max blocks for this example
#define THREADS_PER_BLOCK 256 // Max threads per block

// Structure to define a block in the blockchain
struct Block {
    int blockID;
    long timestamp;
    int transactionCount;
    int previousHash;
    int currentHash;
};

// Kernel to calculate block hashes
__global__ void calculateHash(Block *blocks, int *hashes, int numBlocks) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < numBlocks) {
        hashes[index] = blocks[index].previousHash ^ blocks[index].transactionCount + blocks[index].timestamp;
    }
}

// Kernel to count the number of blocks in the blockchain
__global__ void countBlocks(int *count, int numBlocks) {
    __shared__ int countCache[THREADS_PER_BLOCK];
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    countCache[threadIdx.x] = (index < numBlocks) ? 1 : 0;
    __syncthreads();

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        if (threadIdx.x < offset) {
            countCache[threadIdx.x] += countCache[threadIdx.x + offset];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(count, countCache[0]);
    }
}

// Function to initialize a dataset of blocks
void initializeBlocks(Block *blocks, int numBlocks) {
    for (int i = 0; i < numBlocks; i++) {
        blocks[i].blockID = i;
        blocks[i].timestamp = 1620000000 + i * 10;
        blocks[i].transactionCount = (i % 10) + 1;
        blocks[i].previousHash = i * 31;
        blocks[i].currentHash = 0;
    }
}

// Function to print the blocks
void printBlocks(const Block *blocks, int numBlocks) {
    printf("BlockID\tTimestamp\tTransactions\tPrevHash\tCurrHash\n");
    for (int i = 0; i < numBlocks; i++) {
        printf("%d\t%ld\t%d\t%d\t%d\n",
               blocks[i].blockID,
               blocks[i].timestamp,
               blocks[i].transactionCount,
               blocks[i].previousHash,
               blocks[i].currentHash);
    }
}

int main() {
    int numBlocks = MAX_BLOCKS;
    int *d_count;
    int host_count = 0;
    Block *blocks = (Block*)malloc(numBlocks * sizeof(Block));

    initializeBlocks(blocks, numBlocks);

    Block *d_blocks;
    int *d_hashes;
    cudaMalloc((void**)&d_blocks, numBlocks * sizeof(Block));
    cudaMalloc((void**)&d_hashes, numBlocks * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));

     FILE *outputFile = fopen("output.txt", "w");
           if (outputFile) {
    fprintf(outputFile, "Blockchain Simulation:\n");
    for (int i = 0; i < numBlocks; i++) {
        fprintf(outputFile, "%d\t%ld\t%d\t%d\t%d\n", blocks[i].blockID, blocks[i].timestamp, blocks[i].transactionCount, blocks[i].previousHash, blocks[i].currentHash);
        }
    fprintf(outputFile, "\nTotal number of blocks: %d\n", host_count);
    fclose(outputFile);
     }

    cudaMemcpy(d_blocks, blocks, numBlocks * sizeof(Block), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &host_count, sizeof(int), cudaMemcpyHostToDevice);

    int blocksPerGrid = (numBlocks + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    calculateHash<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_blocks, d_hashes, numBlocks);
    cudaDeviceSynchronize();

    countBlocks<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_count, numBlocks);
    cudaDeviceSynchronize();

    cudaMemcpy(&host_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(blocks, d_blocks, numBlocks * sizeof(Block), cudaMemcpyDeviceToHost);

    printf("Blockchain Simulation:\n");
    printBlocks(blocks, numBlocks);


    cudaFree(d_blocks);
    cudaFree(d_hashes);
    cudaFree(d_count);
    free(blocks);

    return 0;
}


Overwriting blockchain_simulation.cu


In [15]:
!nvcc blockchain_simulation.cu -o blockchain_simulation


In [16]:
!./blockchain_simulation


Blockchain Simulation:
BlockID	Timestamp	Transactions	PrevHash	CurrHash
0	1620000000	1	0	0
1	1620000010	2	31	0
2	1620000020	3	62	0
3	1620000030	4	93	0
4	1620000040	5	124	0
5	1620000050	6	155	0
6	1620000060	7	186	0
7	1620000070	8	217	0
8	1620000080	9	248	0
9	1620000090	10	279	0
10	1620000100	1	310	0
11	1620000110	2	341	0
12	1620000120	3	372	0
13	1620000130	4	403	0
14	1620000140	5	434	0
15	1620000150	6	465	0
16	1620000160	7	496	0
17	1620000170	8	527	0
18	1620000180	9	558	0
19	1620000190	10	589	0
20	1620000200	1	620	0
21	1620000210	2	651	0
22	1620000220	3	682	0
23	1620000230	4	713	0
24	1620000240	5	744	0
25	1620000250	6	775	0
26	1620000260	7	806	0
27	1620000270	8	837	0
28	1620000280	9	868	0
29	1620000290	10	899	0
30	1620000300	1	930	0
31	1620000310	2	961	0
32	1620000320	3	992	0
33	1620000330	4	1023	0
34	1620000340	5	1054	0
35	1620000350	6	1085	0
36	1620000360	7	1116	0
37	1620000370	8	1147	0
38	1620000380	9	1178	0
39	1620000390	10	1209	0
40	1620000400	1	1240	0
41	1620000410	2	1271	0
42	162

In [17]:
!ls -l


total 984
-rwxr-xr-x 1 root root 967504 Nov  7 19:23 blockchain_simulation
-rw-r--r-- 1 root root   3723 Nov  7 19:23 blockchain_simulation.cu
-rw-r--r-- 1 root root  25305 Nov  7 19:23 output.txt
drwxr-xr-x 1 root root   4096 Nov  5 14:22 sample_data


In [18]:
from google.colab import files
files.download("output.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>